# HW1 
### For Homework 1, I will opt to do part II of the homework. Firstly, I will be importing several libraries for my analysis and cleaning the data. The data will be pulled from the yfinance API for 5-years of historical trading data ranging from 2019-08-01 to 2024-08-01. The two commodities for these will be the Crude Oil Oct 2024 futures market and the Natural Gas Oct 2024 futures market.

In [7]:
# Financial libraries
import yfinance as yf
import gs_quant.timeseries.econometrics as gs
from gs_quant.timeseries.econometrics import Returns

# Numerical Libraries
import pandas as pd
from scipy.stats import jarque_bera, skew, kurtosis, norm
import numpy as np
import statsmodels.api

# Plotting Libraries
import plotly.express as px
import plotly.graph_objects as go
import matplotlib
from matplotlib import pyplot as plt

# To ignore warnings
import warnings

In [8]:
warnings.filterwarnings('ignore')

In [9]:
# Dowloading data using yfinance API
crude = yf.download('CL=F', start='2019-01-01', end='2024-08-01')
ng = yf.download('NG=F', start='2019-01-01', end='2024-08-01')

# Cleaning data for analysis
crude.reset_index(inplace=True)
ng.reset_index(inplace=True)

crude.rename(columns={'Adj Close' : 'crude_price'},inplace=True)
ng.rename(columns={'Adj Close' : 'ng_price'}, inplace=True)

prices=crude[['Date','crude_price']]
prices['ng_price']=ng['ng_price']

prices.drop(prices['crude_price'].idxmin(),inplace=True)

# Calculating log returns via gsquant
prices['crude_log_returns']=gs.returns(prices['crude_price'],type=Returns.LOGARITHMIC)
prices['ng_log_returns']=gs.returns(prices['ng_price'],type=Returns.LOGARITHMIC)

prices.fillna(0,inplace=True)

prices.head()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


,Date,crude_price,ng_price,crude_log_returns,ng_log_returns
0,2019-01-02,46.540001,2.958,0.000000,0.000000
1,2019-01-03,47.090000,2.945,0.011748,-0.004405
2,2019-01-04,47.959999,3.044,0.018307,0.033064
3,2019-01-07,48.520000,2.944,0.011609,-0.033403
4,2019-01-08,49.779999,2.967,0.025637,0.007782


In [22]:
# Summary Statistics
summary=prices[['crude_price','ng_price','crude_log_returns','ng_log_returns']].describe()

# Skewness
crude_skew = skew(prices['crude_price'])
ng_skew = skew(prices['ng_price'])
crude_ret_skew = skew(prices['crude_log_returns'])
ng_ret_skew = skew(prices['ng_log_returns'])

# Kurtosis
crude_kurt = kurtosis(prices['crude_price'])
ng_kurt = kurtosis(prices['ng_price'])
crude_ret_kurt = kurtosis(prices['crude_log_returns'])
ng_ret_kurt = kurtosis(prices['ng_log_returns'])

# Adding to dataframe
summary.loc[len(summary.index)] = [crude_skew,ng_skew,crude_ret_skew,ng_ret_skew] 
summary.rename(index={len(summary.index) - 1: 'skew'}, inplace=True)

summary.loc[len(summary.index)] = [crude_kurt,ng_kurt,crude_ret_kurt,ng_ret_kurt] 
summary.rename(index={len(summary.index) - 1: 'kurt'}, inplace=True)


display(summary)

,crude_price,ng_price,crude_log_returns,ng_log_returns
count,1404.000000,1404.000000,1404.000000,1404.000000
mean,68.536610,3.380736,0.000367,-0.000266
std,19.685614,1.784719,0.036347,0.044395
min,10.010000,1.482000,-0.601676,-0.300480
25%,55.687499,2.293250,-0.012733,-0.024634
50%,71.115002,2.675000,0.002174,0.000000
75%,80.837502,3.840500,0.014939,0.023820
max,123.699997,9.680000,0.319634,0.381727
skew,-0.083146,1.623184,-3.168146,0.220925
kurt,0.078142,1.840367,68.588116,6.775662


In [ ]:
# Skewness
crude_skew = skew(prices['crude_log_returns'])
ng_skew = skew(prices['ng_log_returns'])

# Kurtosis
crude_kurt = kurtosis(prices['crude_log_returns'])
ng_kurt = kurtosis(prices['ng_log_returns'])

In [23]:
px.scatter(prices, x=prices['crude_price'], y=prices['ng_price'], title='Brent Crude Oil vs Henry Hub Natural Gas Daily Log Returns')